This notebook demonstrates core Conv2D structural compression in an aggressive manner.

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import shutil
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
BATCH_SIZE = 128
# You may need to lower this depending upon your GPU

TARGET_SIZE = (96, 96)
# This model uses 96x96 images

In [3]:
# These were created as part of our data preparation notebook
train_storage_ds = "C:/colab/data/birds/cache/train"
valid_storage_ds = "C:/colab/data/birds/cache/valid"
test_storage_ds = "C:/colab/data/birds/cache/test"

In [4]:
# Generator
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    brightness_range=[0.75,1.15],
    rotation_range=30,
    channel_shift_range=10,
    width_shift_range=1.0,
    height_shift_range=1.0,
    shear_range=0.02,
    horizontal_flip=True,
    fill_mode='reflect',
)

# Many folks are against validation augmentation but I use it as the changes
# here are mostly minimal and gives more variation. The test generator is pure.
validation_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    brightness_range=[0.75,1.15],
    rotation_range=30,
    channel_shift_range=10,
    fill_mode='reflect',
    validation_split=0.2,
    horizontal_flip=True,
)


test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
)

In [5]:
train_images = train_generator.flow_from_directory(
    directory=train_storage_ds,
    interpolation='bilinear',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
)

val_images = validation_generator.flow_from_directory(
    directory=valid_storage_ds,
    interpolation='bilinear',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    seed=42,
    shuffle=True,
)

test_images = test_generator.flow_from_directory(
    directory=test_storage_ds,
    interpolation='bilinear',
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 54067 images belonging to 524 classes.
Found 13516 images belonging to 524 classes.
Found 16896 images belonging to 524 classes.


In [7]:
# absolute-cosine regularization
# https://www.amazon.science/publications/quantization-aware-training-with-absolute-cosine-regularization-for-automatic-speech-recognition
class ACosR(tf.keras.regularizers.Regularizer):
    def __init__(self, weight=1.0):
        self.weight = weight

    def __call__(self, weights):
        # Calculate the absolute cosine similarity
        cos_similarity = tf.reduce_mean(tf.abs(tf.reduce_sum(weights * tf.roll(weights, shift=1, axis=-1), axis=-1)))

        # Apply the regularization term to the loss
        regularization_loss = self.weight * cos_similarity

        return regularization_loss

    def get_config(self):
        return {'weight': float(self.weight)}


In [8]:
# Load the pretrained model
pretrained_model = tf.keras.applications.EfficientNetB0(input_shape=(96, 96, 3), include_top=False, weights='imagenet')
pretrained_model.trainable = False
pretrained_model.layers[2]._name = 'normalization'

# Enable ACosR
for i in range(0, len(pretrained_model.layers)):
    layer = pretrained_model.layers[i]
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.DepthwiseConv2D):
        pretrained_model.layers[i].kernel_regularizer = ACosR(weight = 0.005)

for i in range(0, len(pretrained_model.layers)):
    if hasattr(pretrained_model.layers[i], "activation"):
        if "swish" in str(pretrained_model.layers[i].activation):
            pretrained_model.layers[i].activation = tf.keras.activations.relu


x = pretrained_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(200, trainable = False, kernel_regularizer = ACosR(weight = 0.005), name="dense_1", activation='relu')(x)
x = Dense(256, trainable = False, kernel_regularizer = ACosR(weight = 0.005), name="dense_2", activation='relu')(x)
outputs = Dense(524, kernel_regularizer = ACosR(weight = 0.005), trainable = False, activation=tf.keras.activations.softmax, name="dense_output")(x)
model = Model(inputs=pretrained_model.input, outputs=outputs)
model.load_weights('C:/colab/data/birds/saves/96x96_524_full_relu/saved_model')

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 rescaling (Rescaling)          (None, 96, 96, 3)    0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 96, 96, 3)    7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 96, 96, 3)    0           ['normalization[0][0]']          
                                                                                              

In [9]:
# All layers are locked but running an epoch for demonstration purposes
model.compile(
    optimizer=Adam(1e-6),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=1,
)

423/423 [==============================] - 120s 274ms/step - loss: 0.0339 - accuracy: 0.9911 - val_loss: 0.5087 - val_accuracy: 0.9065


In [10]:
# Saving to reload partially
model.save('C:/colab/data/birds/525-demonstration-model-aggressive-compression.keras')

In [13]:
# Aggressive Model Compression
pretrained_model = tf.keras.applications.EfficientNetB0(input_shape=(96, 96, 3), include_top=False, weights='imagenet')
pretrained_model.trainable = False
pretrained_model.layers[2]._name = 'normalization'
selected_layer_index = 227

# Giving block6d focus along with the new dense
for i in range(214, len(pretrained_model.layers)):
    pretrained_model.layers[i].trainable = True

# Enable ACosR
for i in range(0, len(pretrained_model.layers)):
    layer = pretrained_model.layers[i]
    if isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.DepthwiseConv2D):
        pretrained_model.layers[i].kernel_regularizer = ACosR(weight = 0.005)

for i in range(0, len(pretrained_model.layers)):
    if hasattr(pretrained_model.layers[i], "activation"):
        if "swish" in str(pretrained_model.layers[i].activation):
            pretrained_model.layers[i].activation = tf.keras.activations.relu

selected_layer_output = pretrained_model.layers[selected_layer_index].output
x = selected_layer_output
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Invalidated as you'll see in the warning text, 1280 -> 1152
x = Dense(200, trainable = True, kernel_regularizer = ACosR(weight = 0.005), name="dense_1", activation='relu')(x)
x = Dense(256, trainable = False, kernel_regularizer = ACosR(weight = 0.005), name="dense_2", activation='relu')(x)
outputs = Dense(524, kernel_regularizer = ACosR(weight = 0.005), trainable = False, activation=tf.keras.activations.softmax, name="dense_output")(x)
model = Model(inputs=pretrained_model.input, outputs=outputs)
model.load_weights('C:/colab/data/birds/525-demonstration-model-aggressive-compression.keras', skip_mismatch=True, by_name=True)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 rescaling_3 (Rescaling)        (None, 96, 96, 3)    0           ['input_4[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 96, 96, 3)    7           ['rescaling_3[0][0]']            
                                                                                                  
 tf.math.truediv_3 (TFOpLambda)  (None, 96, 96, 3)   0           ['normalization[0][0]']          
                                                                                            

In [15]:
# "Compress" -- here we'll run one epoch to demonstrate the relearning process
# The model will require several epochs.
model.compile(
    optimizer=Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=1,
)

423/423 [==============================] - 117s 271ms/step - loss: 0.7256 - accuracy: 0.8411 - val_loss: 0.5867 - val_accuracy: 0.8693


In [16]:
# Now let's evaluate the test images for good measure (despite only having run one epoch)
results = model.evaluate(test_images, verbose=1)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

132/132 [==============================] - 11s 82ms/step - loss: 0.6359 - accuracy: 0.8555
    Test Loss: 0.63587
Test Accuracy: 85.55%
